In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import pandas as pd

Your DataFrame<br>
df = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with the actual CSV file path

List of facility codes

In [ ]:
facility_codes_to_apply = [7198, 5645, 90, 29, 2586, 2492, 6958, 3228, 21, 3232, 2498, 1388, 45, 3666, 2497, 36, 5743, 120936, 2464]
facility_codes_to_apply_70 = [213, 5719, 1163, 2541, 10267, 5668, 8697, 282, 5617, 1198, 475744, 10941, 70, 205062, 183, 494582, 5699]

Function to apply to each row

In [ ]:
def apply_new_column(row):
    if row['facility_code'] in facility_codes_to_apply:
        return 2492
    elif row['facility_code'] in facility_codes_to_apply_70:
        return 70  # Or any other value you prefer for non-matching facility codes
    else:
        return 488965

Apply the function to create the new column

In [ ]:
aug_pred_prob_byrd_70_epic['main_facility'] = aug_pred_prob_byrd_70_epic.apply(apply_new_column, axis=1)
# Your DataFrame
# aug_pred_prob_byrd_70_epic = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with the actual CSV file path

Initialize an empty list to store data for different thresholds

In [ ]:
thresholds = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
data_list = []

Iterate over unique facilities

In [ ]:
for facility in aug_pred_prob_byrd_70_epic['main_facility'].unique():
    facility_df = aug_pred_prob_byrd_70_epic[aug_pred_prob_byrd_70_epic['main_facility'] == facility]
    
    for threshold in thresholds:
        column_name = f'result_thr_{int(threshold * 100)}'
    
        # Apply the threshold to create a new column in the filtered DataFrame
        facility_df[column_name] = facility_df.apply(lambda row: row['predicted_class'] if row['max_prob'] > threshold else 0, axis=1)
        
        # Filter the DataFrame based on the new column
        test_threshold = facility_df[facility_df[column_name] != 0]
        
        # Calculate classification report and confusion matrix
        report = classification_report(test_threshold['EANDM_Final_code_1'], test_threshold[column_name], output_dict=True)
        
        # Create a data dictionary for this threshold
        data = {
            'facility': [facility],
            'Threshold': [threshold],
            'Accuracy': [report['accuracy']],
            'Coverage': [len(test_threshold) / len(facility_df)],
            'Total': [len(facility_df)]
        }
                # Add 'macro avg' and 'weighted avg' to the data dictionary
        data['Support'] = [report['macro avg']['support']]
        data['macro avg_F1_Score'] = [report['macro avg']['f1-score']]
        data['weighted avg_F1_Score'] = [report['weighted avg']['f1-score']]
    
        # Extract precision, recall, f1-score, and support for each class
        for class_name in report.keys():
            if class_name not in ['accuracy', 'weighted avg', 'macro avg']:
                data[class_name + '_Precision'] = [report[class_name]['precision']]
                data[class_name + '_Recall'] = [report[class_name]['recall']]
                data[class_name + '_F1_Score'] = [report[class_name]['f1-score']]
                data[class_name + '_Support'] = [report[class_name]['support']]
        
        
        # Append the data for this threshold to the list
        data_list.append(data)

Create the final DataFrame by concatenating the data for different thresholds

In [ ]:
df = pd.concat([pd.DataFrame(data) for data in data_list], ignore_index=True)

Display the final DataFrame

In [ ]:
df